<a href="https://colab.research.google.com/github/BrunoFerCam/bertoti/blob/main/botPeixaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyTelegramBotAPI transformers torch wikipedia-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.0/245.0 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
import telebot
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import wikipediaapi

In [8]:
model_name = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [34]:
API_TOKEN = '7469665499:AAHPUUNbXgmh6bJXPKgXb5asplhCfaJA9kM'
bot = telebot.TeleBot(API_TOKEN)

In [31]:
knowledge_base = {
    "store_hours": "Our store is open from 9 AM to 7 PM, Monday to Saturday. On Sundays, we are open from 10 AM to 4 PM. We are closed on public holidays.",
    "discounts": "We currently have a 15% discount on all fish tanks over 50 gallons. We also offer a 10% discount on fish food and accessories during the weekend sale.",
    "product_prices": {
        "20_gallon_tank": "$50",
        "50_gallon_tank": "$100",
        "filter": "$20",
        "fish_food": "$10",
        "fish_net": "$5",
        "aquarium_light": "$25"
    },
    "product_availability": {
        "goldfish": "Available",
        "betta_fish": "Out of stock",
        "neon_tetra": "Available",
        "guppy": "Available",
        "plecostomus": "Available"
    },
    "services": {
        "water_testing": "We offer free water testing for your aquarium. Bring a sample and get results in 10 minutes.",
        "tank_setup": "We provide professional tank setup services starting at $50. This includes equipment installation and initial water conditioning.",
        "fish_health_check": "Get a free fish health check-up every first Saturday of the month."
    },
    "store_policies": {
        "returns": "Items can be returned within 30 days of purchase with a receipt. Exclusions apply to live animals.",
        "warranty": "We offer a 1-year warranty on all electrical products, including filters and lights.",
        "membership": "Join our membership program for $20/year and receive exclusive discounts and early access to sales."
    },
    "frequently_asked_questions": {
        "how_to_care_for_goldfish": "Goldfish require a minimum of a 20-gallon tank, clean water, and a balanced diet. Regular tank maintenance is essential for their health.",
        "best_fish_for_beginners": "Some of the best fish for beginners include goldfish, betta fish, and guppies due to their hardy nature and ease of care."
    }
}


In [35]:
def fetch_knowledge_base_response(user_input):
    user_input = user_input.lower()

    if "store hours" in user_input or "open" in user_input:
        return knowledge_base["store_hours"]
    elif "discount" in user_input or "sale" in user_input:
        return knowledge_base["discounts"]
    elif "price" in user_input or "cost" in user_input:
        if "20 gallon tank" in user_input:
            return f"The price of a 20-gallon tank is {knowledge_base['product_prices']['20_gallon_tank']}."
        elif "50 gallon tank" in user_input:
            return f"The price of a 50-gallon tank is {knowledge_base['product_prices']['50_gallon_tank']}."
        elif "filter" in user_input:
            return f"The price of a filter is {knowledge_base['product_prices']['filter']}."
        elif "fish food" in user_input:
            return f"The price of fish food is {knowledge_base['product_prices']['fish_food']}."
        elif "fish net" in user_input:
            return f"The price of a fish net is {knowledge_base['product_prices']['fish_net']}."
        elif "aquarium light" in user_input:
            return f"The price of an aquarium light is {knowledge_base['product_prices']['aquarium_light']}."
    elif "available" in user_input or "stock" in user_input:
        if "goldfish" in user_input:
            return f"Goldfish are {knowledge_base['product_availability']['goldfish']}."
        elif "betta" in user_input:
            return f"Betta fish are {knowledge_base['product_availability']['betta_fish']}."
        elif "neon tetra" in user_input:
            return f"Neon tetras are {knowledge_base['product_availability']['neon_tetra']}."
        elif "guppy" in user_input:
            return f"Guppies are {knowledge_base['product_availability']['guppy']}."
        elif "plecostomus" in user_input:
            return f"Plecostomus are {knowledge_base['product_availability']['plecostomus']}."
        elif "blue eyes white fish" in user_input:
            return f"Blue eyes white fish are the ultimate engines of destruction."
    elif "service" in user_input:
        if "water testing" in user_input:
            return knowledge_base["services"]["water_testing"]
        elif "tank setup" in user_input:
            return knowledge_base["services"]["tank_setup"]
        elif "health check" in user_input:
            return knowledge_base["services"]["fish_health_check"]
    elif "policy" in user_input:
        if "return" in user_input:
            return knowledge_base["store_policies"]["returns"]
        elif "warranty" in user_input:
            return knowledge_base["store_policies"]["warranty"]
        elif "membership" in user_input:
            return knowledge_base["store_policies"]["membership"]
    elif "faq" in user_input or "frequently asked question" in user_input:
        if "care for goldfish" in user_input:
            return knowledge_base["frequently_asked_questions"]["how_to_care_for_goldfish"]
        elif "best fish for beginners" in user_input:
            return knowledge_base["frequently_asked_questions"]["best_fish_for_beginners"]
    elif "fish" in user_input:
        fish_name = user_input.replace("fish", "").strip()
        return fetch_wikipedia_summary(fish_name)

    return "I'm not sure about that. Can you please provide more details?"
def fetch_wikipedia_summary(fish_name):
    wiki_wiki = wikipediaapi.Wikipedia('LambariStore/1.0')
    page = wiki_wiki.page(fish_name)
    if page.exists():
        return page.summary
    else:
        return "Sorry, I couldn't find information on that fish."





@bot.message_handler(func=lambda message: True)
def handle_message(message):
    user_input = message.text

    if "fish" in user_input.lower():
        fish_name = user_input.lower().replace("fish", "").strip()
        wiki_response = fetch_wikipedia_summary(fish_name)
        bot.reply_to(message, wiki_response)
    else:
        kb_response = fetch_knowledge_base_response(user_input)
        if kb_response:
            bot.reply_to(message, kb_response)
        else:
            inputs = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
            outputs = model.generate(inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)

            response = tokenizer.decode(outputs[:, inputs.shape[-1]:][0], skip_special_tokens=True)
            bot.reply_to(message, response)

In [ ]:
bot.polling()